In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

C:\Users\jains\anaconda3\envs\DL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [6]:
mnist = torchvision.datasets.MNIST(root='../data',
                                  train=True,
                                  transform=transform,
                                  download=False)

In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                         batch_size=batch_size,
                                         shuffle=True)

In [8]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

In [9]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

In [10]:
D = D.to(device)
G = G.to(device)

In [11]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [12]:
def denorm(x):
    x = (x + 1) / 2
    return x.clamp(0, 1)

In [13]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        #training discriminator
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels).to(device)
        fake_score = outputs
        
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        #training generator
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels)
        
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
                 .format(epoch, num_epochs, (i+1), total_step, d_loss.item(), g_loss.item(),
                        real_score.mean().item(), fake_score.mean().item()))
            
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch: [0/200], Step: [200/600], d_loss: 0.0639, g_loss: 3.9627, D(x): 0.99, D(G(z)): 0.05
Epoch: [0/200], Step: [400/600], d_loss: 0.0869, g_loss: 5.8750, D(x): 0.98, D(G(z)): 0.07
Epoch: [0/200], Step: [600/600], d_loss: 0.0417, g_loss: 5.7428, D(x): 0.99, D(G(z)): 0.04
Epoch: [1/200], Step: [200/600], d_loss: 0.0370, g_loss: 5.3478, D(x): 0.99, D(G(z)): 0.02
Epoch: [1/200], Step: [400/600], d_loss: 0.7522, g_loss: 3.5601, D(x): 0.93, D(G(z)): 0.43
Epoch: [1/200], Step: [600/600], d_loss: 0.0516, g_loss: 4.9988, D(x): 0.98, D(G(z)): 0.03
Epoch: [2/200], Step: [200/600], d_loss: 0.8775, g_loss: 5.3417, D(x): 0.95, D(G(z)): 0.52
Epoch: [2/200], Step: [400/600], d_loss: 0.0689, g_loss: 4.7940, D(x): 0.97, D(G(z)): 0.02
Epoch: [2/200], Step: [600/600], d_loss: 0.5491, g_loss: 4.1219, D(x): 0.84, D(G(z)): 0.11
Epoch: [3/200], Step: [200/600], d_loss: 0.2822, g_loss: 3.4364, D(x): 0.90, D(G(z)): 0.11
Epoch: [3/200], Step: [400/600], d_loss: 0.1533, g_loss: 3.5713, D(x): 0.95, D(G(z)): 0.07

Epoch: [30/200], Step: [200/600], d_loss: 0.3467, g_loss: 3.4978, D(x): 0.97, D(G(z)): 0.19
Epoch: [30/200], Step: [400/600], d_loss: 0.2188, g_loss: 4.1339, D(x): 0.91, D(G(z)): 0.06
Epoch: [30/200], Step: [600/600], d_loss: 0.4027, g_loss: 3.3481, D(x): 0.89, D(G(z)): 0.15
Epoch: [31/200], Step: [200/600], d_loss: 0.3843, g_loss: 4.3973, D(x): 0.87, D(G(z)): 0.07
Epoch: [31/200], Step: [400/600], d_loss: 0.4525, g_loss: 3.2572, D(x): 0.90, D(G(z)): 0.19
Epoch: [31/200], Step: [600/600], d_loss: 0.3752, g_loss: 3.1565, D(x): 0.89, D(G(z)): 0.13
Epoch: [32/200], Step: [200/600], d_loss: 0.3507, g_loss: 4.4374, D(x): 0.92, D(G(z)): 0.13
Epoch: [32/200], Step: [400/600], d_loss: 0.3729, g_loss: 3.2325, D(x): 0.94, D(G(z)): 0.18
Epoch: [32/200], Step: [600/600], d_loss: 0.2780, g_loss: 4.6911, D(x): 0.94, D(G(z)): 0.13
Epoch: [33/200], Step: [200/600], d_loss: 0.2623, g_loss: 3.8613, D(x): 0.88, D(G(z)): 0.05
Epoch: [33/200], Step: [400/600], d_loss: 0.5654, g_loss: 3.8203, D(x): 0.91, D(

Epoch: [60/200], Step: [200/600], d_loss: 0.6588, g_loss: 2.1483, D(x): 0.81, D(G(z)): 0.24
Epoch: [60/200], Step: [400/600], d_loss: 0.5649, g_loss: 3.0146, D(x): 0.83, D(G(z)): 0.19
Epoch: [60/200], Step: [600/600], d_loss: 0.8359, g_loss: 2.1694, D(x): 0.69, D(G(z)): 0.19
Epoch: [61/200], Step: [200/600], d_loss: 0.7160, g_loss: 1.7874, D(x): 0.80, D(G(z)): 0.27
Epoch: [61/200], Step: [400/600], d_loss: 0.7798, g_loss: 1.7198, D(x): 0.84, D(G(z)): 0.34
Epoch: [61/200], Step: [600/600], d_loss: 0.6672, g_loss: 2.2778, D(x): 0.84, D(G(z)): 0.26
Epoch: [62/200], Step: [200/600], d_loss: 0.6304, g_loss: 2.3186, D(x): 0.88, D(G(z)): 0.28
Epoch: [62/200], Step: [400/600], d_loss: 1.0873, g_loss: 1.8495, D(x): 0.88, D(G(z)): 0.47
Epoch: [62/200], Step: [600/600], d_loss: 0.6758, g_loss: 2.2382, D(x): 0.78, D(G(z)): 0.22
Epoch: [63/200], Step: [200/600], d_loss: 0.5242, g_loss: 2.4218, D(x): 0.85, D(G(z)): 0.20
Epoch: [63/200], Step: [400/600], d_loss: 0.7657, g_loss: 2.6138, D(x): 0.77, D(

Epoch: [90/200], Step: [200/600], d_loss: 0.6967, g_loss: 1.9343, D(x): 0.73, D(G(z)): 0.19
Epoch: [90/200], Step: [400/600], d_loss: 0.7572, g_loss: 1.7989, D(x): 0.77, D(G(z)): 0.27
Epoch: [90/200], Step: [600/600], d_loss: 0.6122, g_loss: 1.8931, D(x): 0.77, D(G(z)): 0.20
Epoch: [91/200], Step: [200/600], d_loss: 0.6923, g_loss: 1.9602, D(x): 0.82, D(G(z)): 0.28
Epoch: [91/200], Step: [400/600], d_loss: 0.9476, g_loss: 1.9705, D(x): 0.66, D(G(z)): 0.26
Epoch: [91/200], Step: [600/600], d_loss: 0.7617, g_loss: 1.8774, D(x): 0.78, D(G(z)): 0.28
Epoch: [92/200], Step: [200/600], d_loss: 0.9551, g_loss: 2.1338, D(x): 0.68, D(G(z)): 0.26
Epoch: [92/200], Step: [400/600], d_loss: 0.9240, g_loss: 2.1099, D(x): 0.69, D(G(z)): 0.23
Epoch: [92/200], Step: [600/600], d_loss: 0.6863, g_loss: 1.8117, D(x): 0.78, D(G(z)): 0.25
Epoch: [93/200], Step: [200/600], d_loss: 0.8713, g_loss: 2.3553, D(x): 0.72, D(G(z)): 0.27
Epoch: [93/200], Step: [400/600], d_loss: 0.8603, g_loss: 1.7282, D(x): 0.83, D(

Epoch: [119/200], Step: [600/600], d_loss: 0.7791, g_loss: 1.5173, D(x): 0.82, D(G(z)): 0.35
Epoch: [120/200], Step: [200/600], d_loss: 1.0115, g_loss: 1.5947, D(x): 0.63, D(G(z)): 0.27
Epoch: [120/200], Step: [400/600], d_loss: 1.0662, g_loss: 1.2867, D(x): 0.71, D(G(z)): 0.36
Epoch: [120/200], Step: [600/600], d_loss: 0.9495, g_loss: 1.6605, D(x): 0.73, D(G(z)): 0.36
Epoch: [121/200], Step: [200/600], d_loss: 0.8931, g_loss: 1.5884, D(x): 0.73, D(G(z)): 0.32
Epoch: [121/200], Step: [400/600], d_loss: 0.9303, g_loss: 1.6190, D(x): 0.66, D(G(z)): 0.29
Epoch: [121/200], Step: [600/600], d_loss: 0.9152, g_loss: 1.5465, D(x): 0.67, D(G(z)): 0.23
Epoch: [122/200], Step: [200/600], d_loss: 0.9024, g_loss: 1.9049, D(x): 0.74, D(G(z)): 0.31
Epoch: [122/200], Step: [400/600], d_loss: 1.0102, g_loss: 1.3207, D(x): 0.66, D(G(z)): 0.31
Epoch: [122/200], Step: [600/600], d_loss: 1.1390, g_loss: 1.8385, D(x): 0.77, D(G(z)): 0.44
Epoch: [123/200], Step: [200/600], d_loss: 0.7341, g_loss: 1.6558, D(x

Epoch: [149/200], Step: [400/600], d_loss: 0.8084, g_loss: 1.7206, D(x): 0.70, D(G(z)): 0.26
Epoch: [149/200], Step: [600/600], d_loss: 0.7721, g_loss: 1.8657, D(x): 0.78, D(G(z)): 0.31
Epoch: [150/200], Step: [200/600], d_loss: 0.7818, g_loss: 1.6491, D(x): 0.75, D(G(z)): 0.31
Epoch: [150/200], Step: [400/600], d_loss: 0.8304, g_loss: 1.3856, D(x): 0.75, D(G(z)): 0.34
Epoch: [150/200], Step: [600/600], d_loss: 0.8650, g_loss: 1.2129, D(x): 0.75, D(G(z)): 0.35
Epoch: [151/200], Step: [200/600], d_loss: 1.0062, g_loss: 1.7162, D(x): 0.70, D(G(z)): 0.33
Epoch: [151/200], Step: [400/600], d_loss: 1.0807, g_loss: 1.1694, D(x): 0.66, D(G(z)): 0.33
Epoch: [151/200], Step: [600/600], d_loss: 0.8847, g_loss: 1.9004, D(x): 0.70, D(G(z)): 0.27
Epoch: [152/200], Step: [200/600], d_loss: 0.7922, g_loss: 1.7552, D(x): 0.77, D(G(z)): 0.31
Epoch: [152/200], Step: [400/600], d_loss: 0.8693, g_loss: 1.6789, D(x): 0.77, D(G(z)): 0.37
Epoch: [152/200], Step: [600/600], d_loss: 1.0194, g_loss: 1.2124, D(x

Epoch: [179/200], Step: [200/600], d_loss: 0.9264, g_loss: 1.4543, D(x): 0.70, D(G(z)): 0.30
Epoch: [179/200], Step: [400/600], d_loss: 0.8972, g_loss: 1.7981, D(x): 0.71, D(G(z)): 0.31
Epoch: [179/200], Step: [600/600], d_loss: 0.9403, g_loss: 1.5422, D(x): 0.72, D(G(z)): 0.32
Epoch: [180/200], Step: [200/600], d_loss: 1.0564, g_loss: 0.9034, D(x): 0.72, D(G(z)): 0.38
Epoch: [180/200], Step: [400/600], d_loss: 0.9864, g_loss: 1.5971, D(x): 0.67, D(G(z)): 0.32
Epoch: [180/200], Step: [600/600], d_loss: 0.8590, g_loss: 1.2061, D(x): 0.69, D(G(z)): 0.28
Epoch: [181/200], Step: [200/600], d_loss: 0.7813, g_loss: 1.4716, D(x): 0.75, D(G(z)): 0.31
Epoch: [181/200], Step: [400/600], d_loss: 0.7671, g_loss: 2.1021, D(x): 0.70, D(G(z)): 0.21
Epoch: [181/200], Step: [600/600], d_loss: 1.0149, g_loss: 1.7150, D(x): 0.63, D(G(z)): 0.29
Epoch: [182/200], Step: [200/600], d_loss: 0.9301, g_loss: 1.3151, D(x): 0.70, D(G(z)): 0.34
Epoch: [182/200], Step: [400/600], d_loss: 1.0691, g_loss: 1.5168, D(x